<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F%20%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0%20%E2%84%96%E2%80%AF3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Практическая работа № 3  
**Тема.** Классификация текста с помощью классических методов машинного обучения

---

## 1. Цель и задачи работы

**Цель работы** — формирование у обучающегося системного и практического представления о применении классических методов машинного обучения для решения задач текстовой классификации. Работа направлена на развитие навыков проектирования ML-пайплайнов, оценки влияния предварительной обработки и векторизации текста на качество модели, а также на освоение практик интерпретации предсказаний и публикации результатов.

**Основные задачи работы:**

1. Использовать корпус текстов, сформированный в рамках Практической работы № 1, в качестве экспериментальной базы.  
2. Выполнить разметку корпуса для трёх типов задач: **бинарной**, **многоклассовой** и **многометочной классификации**.  
3. Реализовать и сравнить классические методы машинного обучения: **логистическую регрессию**, **метод опорных векторов (SVM)**, **градиентный бустинг (CatBoost, XGBoost)**.  
4. Провести контролируемый эксперимент, оценивающий влияние:  
   - методов токенизации и нормализации (из Практической работы № 1),  
   - методов векторизации (из Практической работы № 2),  
   - происхождения моделей — **самостоятельно обученных** (BPE, Word2Vec и др.) и **внешних предобученных** (например, FastText от Meta).  
5. Применить методы коррекции дисбаланса классов и строгой оценки качества.  
6. Проанализировать интерпретируемость моделей с использованием **SHAP** и **LIME**.  
7. Разработать интерактивный веб-интерфейс для демонстрации и сравнения работы классификаторов.  
8. Обеспечить воспроизводимость результатов через публикацию кода и документацию.

---

## 2. Теоретические предпосылки

Классификация текста — фундаментальная задача машинного обучения, лежащая в основе множества прикладных NLP-систем: анализа тональности, тематической категоризации, фильтрации спама, определения жанра и др. Все эти задачи формально сводятся к одной из трёх постановок:

1. **Бинарная классификация** — выбор одной из двух меток (например, «позитивный» / «негативный» отзыв).  
2. **Многоклассовая классификация** — выбор одной метки из множества (например, «политика», «экономика», «спорт»).  
3. **Многометочная классификация** — присвоение документу нескольких меток одновременно.

В настоящей работе рассматриваются следующие **классические методы машинного обучения**, работающие с фиксированным признаковым пространством:

1. **Линейные модели:**  
   - **Логистическая регрессия** — вероятностная модель с L1/L2-регуляризацией;  
   - **Метод опорных векторов (SVM)** — мощный инструмент для разреженных признаковых пространств.

2. **Ансамблевые методы на основе деревьев:**  
   - **Случайный лес (Random Forest)** — устойчивый к шуму и переобучению ансамбль;  
   - **Градиентный бустинг (CatBoost, XGBoost, LightGBM)** — высокопроизводительные алгоритмы с автоматической обработкой категориальных и разреженных признаков.

Качество этих моделей напрямую зависит от того, **как текст преобразуется в признаковое пространство**. В настоящей работе это пространство формируется на основе:

- **Токенизации**, включая подсловные модели (BPE, WordPiece), обученные вами;  
- **Векторизации**, включая эмбеддинги (Word2Vec, FastText, GloVe), также обученные вами, а также внешние предобученные модели (например, от Meta или Stanford).

Такой подход позволяет оценить, насколько важна **адаптация представления текста к домену** по сравнению с использованием универсальных решений.

---

## 3. Методика и порядок выполнения работы

Практическая работа выполняется последовательно по восьми этапам.

### 3.1. Этап 1. Подготовка экспериментального корпуса

**Задача:** Использовать корпус текстов, созданный в рамках Практической работы № 1, в качестве основы для всех экспериментов.

**Требования к корпусу:**

- **Источники:**  
  Основной — русскоязычные новостные порталы (ТАСС, РИА Новости, Lenta.ru, Meduza, Коммерсант и др.).  
  Допускается включение материалов на языках народов Российской Федерации (татарский, башкирский и др.) для лингвистического разнообразия.

- **Разметка:**  
  - **Бинарная классификация**: тональность (позитив/негатив);  
  - **Многоклассовая**: темы (политика, экономика, спорт, культура);  
  - **Многометочная**: несколько тем на документ.

- **Объём:** не менее 10 000 размеченных документов на тип задачи.  
- **Структура документа:** поля `title`, `text`, `sentiment`, `category`, `categories`.  
- **Формат:** JSONL (по одному документу на строку).  
- **Разделение:** train/validation/test в соотношении 70/15/15 с сохранением стратификации.

> **Примечание:** Корпус уже содержит предобработанный текст из Работы № 1, что позволяет сосредоточиться на этапе классификации.

### 3.2. Этап 2. Формирование признаковых представлений

**Задача:** Создать несколько независимых признаковых пространств на основе различных пайплайнов предобработки.

**Варианты пайплайнов:**

- **Токенизация:**  
  - Наивная (по пробелам);  
  - Лемматизация (`pymorphy2`);  
  - **Подсловные модели, обученные вами** (BPE, WordPiece — из Работы № 1).

- **Векторизация:**  
  - **Статистические методы**: TF-IDF с n-граммами (1–3);  
  - **Эмбеддинги, обученные вами**: Word2Vec, FastText, GloVe (из Работы № 2);  
  - **Внешние предобученные эмбеддинги**: например, `cc.ru.300.vec` от Meta;  
  - Базовый пайплайн: `TfidfVectorizer` с параметрами по умолчанию.

Для эмбеддингов вектор документа формируется как **усреднение векторов всех токенов**.

### 3.3. Этап 3. Реализация классических методов классификации

**Задача:** Разработать модуль для классических ML-моделей.

**Требования к реализации:**

- Поддержка следующих алгоритмов:  
  - Логистическая регрессия (`sklearn.linear_model.LogisticRegression`);  
  - Линейный SVM (`sklearn.svm.LinearSVC`);  
  - CatBoost (`catboost.CatBoostClassifier`);  
  - XGBoost (`xgboost.XGBClassifier`).

- Код оформляется в виде модуля `classical_classifiers.py`.  
- Поддержка автоматической обработки разреженных матриц и плотных эмбеддингов.

### 3.4. Этап 4. Сравнительный анализ методов классификации

**Задача:** Провести эмпирическое сравнение эффективности моделей и пайплайнов.

**Метрики оценки:**

- **Бинарная классификация**: Accuracy, Precision, Recall, F1, ROC-AUC, PR-AUC;  
- **Многоклассовая**: macro/micro F1, Accuracy, Confusion Matrix;  
- **Многометочная**: Subset Accuracy, Hamming Loss, macro/micro F1.

**Форма представления результатов:**  
Сводная таблица `classification_metrics.csv` с разбивкой по типу задачи, пайплайну и модели. Дополнительно — графики ROC и Precision-Recall кривых.

### 3.5. Этап 5. Борьба с дисбалансом классов

**Задача:** Обеспечить корректное обучение при неравномерном распределении меток.

**Методы:**

- Взвешивание классов (`class_weight='balanced'`);  
- При необходимости — синтетическая аугментация в пространстве эмбеддингов (SMOTE);  
- Использование метрик, устойчивых к дисбалансу (PR-AUC, F1).

### 3.6. Этап 6. Интерпретация и анализ предсказаний

**Задача:** Проанализировать, почему модель делает то или иное предсказание.

**Методы:**

- Для линейных моделей — анализ весов признаков (наиболее важные слова);  
- Для всех моделей — применение **SHAP** и **LIME** для локального объяснения;  
- Визуализация вклада отдельных токенов в итоговое предсказание;  
- Анализ ошибок: выборка наиболее частых ошибок и их лингвистический разбор.

### 3.7. Этап 7. Разработка веб-интерфейса для анализа классификаторов

**Задача:** Создать интерактивный инструмент для демонстрации работы моделей.

**Функционал:**

- Ввод произвольного текста;  
- Выбор типа задачи (бинарная/многоклассовая/многометочная);  
- Выбор пайплайна (токенизация + векторизация) и модели;  
- Отображение предсказания с вероятностями и объяснением (SHAP/LIME);  
- Сравнение предсказаний нескольких моделей на одном тексте.

**Рекомендуемые технологии:**  
`Streamlit`, `Gradio`, или `Flask`/`FastAPI` + `Plotly`.

### 3.8. Этап 8. Публикация результатов

**Задача:** Обеспечить открытость и воспроизводимость.

**Требования:**

- Публикация кода на GitHub/GitLab с понятной документацией;  
- Размещение веб-приложения (например, через Hugging Face Spaces или Streamlit Cloud);  
- Для каждой модели — описание в формате **Model Card**, включающее:  
  - тип задачи;  
  - пайплайн предобработки;  
  - метрики качества;  
  - пример использования;  
  - лицензию (рекомендуется MIT).

---

## 4. Дополнительные исследовательские задания

1. **Сравнение «самообученных» и «внешних» моделей:**  
   Оцените, насколько модели, обученные на вашем корпусе, превосходят внешние внешние предобученные эмбеддинги, опубликованные исследовательскими организациями и сообществами, такими как Meta (FAIR), Stanford NLP Group, DeepPavlov и т.п.

2. **Влияние подсловной токенизации:**  
   Исследуйте, улучшает ли использование BPE/WordPiece (обученных вами) качество классификации по сравнению с лемматизацией.

3. **Интерпретация как лингвистический инструмент:**  
   Используйте веса логистической регрессии для выявления лексических маркеров тональности в русскоязычных новостях.

4. **Анализ устойчивости к шуму:**  
   Исследуйте, как опечатки и искажения текста влияют на предсказания разных моделей.

---

## 5. Требования к отчёту

Отчёт оформляется в соответствии с **ГОСТ 7.32–2017** и должен включать:

1. Титульный лист (ФИО, учебная группа, дата, подпись преподавателя);  
2. Постановку задачи и цели;  
3. Описание методов и инструментов;  
4. Структуру и характеристики размеченного корпуса;  
5. Результаты экспериментов (таблицы, графики, визуализации);  
6. Сравнительный анализ классических методов машинного обучения и пайплайнов предобработки;  
7. Результаты интерпретации моделей (SHAP, LIME, анализ весов);  
8. Ссылки на:  
   - Исходный код (GitHub/GitLab);  
   - Веб-приложение;  
9. Выводы и рефлексию (какие комбинации «токенизация → векторизация → модель» оказались наиболее эффективными для русского языка; стоит ли обучать свои модели или использовать внешние).

Список использованных источников оформляется в соответствии с **ГОСТ Р 7.0.5–2008**.

---

## 6. Критерии оценивания

| Оценка | Критерии |
|--------|----------|
| **Отлично (5)** | Полное выполнение всех этапов, включая дополнительные задания. Чётко продемонстрировано сравнение «самообученных» и «внешних» моделей. Наличие функционального веб-интерфейса с возможностью интерпретации. Глубокий анализ результатов. Отчёт соответствует ГОСТ. |
| **Хорошо (4)** | Выполнение всех основных этапов (1–8). Наличие веб-интерфейса и отчёта с визуализациями. Корректный сравнительный анализ пайплайнов и моделей. |
| **Удовлетворительно (3)** | Выполнение этапов 1–6. Отчёт содержит описание методов, таблицы метрик и базовые выводы. |
| **Неудовлетворительно (2)** | Отсутствие ключевых этапов (например, разметки, сравнения источников моделей) или отсутствие отчёта. |
